In [1]:
import os, glob, json
import pickle
import pandas as pd
pd.set_option('display.max_colwidth', None)
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import re
import numpy as np
from scipy import stats
from collections import Counter, defaultdict
import random, math
from tqdm import tqdm
import platform

# print machine name
machine_name = platform.node().split(".")[0]
print(machine_name)

# print available scratch directories
print(" ".join(os.listdir(f"/{machine_name}")))
scr_dir = os.listdir(f"/{machine_name}")[0]

DISK_IO_DIR = "/{}/{}/yiweil".format(machine_name,scr_dir)
print(DISK_IO_DIR)
if not os.path.exists(DISK_IO_DIR):
    os.mkdir(DISK_IO_DIR)

john10
scr1
/john10/scr1/yiweil


In [ ]:
import spacy
nlp = spacy.load("en_trf_bertbaseuncased_lg")

In [112]:
from transformers import BertTokenizer, BertModel
import pandas as pd
import numpy as np
import torch
# Loading the pre-trained BERT model
###################################
# Embeddings will be derived from
# the outputs of this model
model = BertModel.from_pretrained('bert-base-uncased',
           output_hidden_states = True,)
# Setting up the tokenizer
###################################
# This is the same tokenizer that
# was used in the model to generate
# embeddings to ensure consistency
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

I1201 11:52:32.998061 139822730319616 configuration_utils.py:283] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /sailhome/yiweil/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
I1201 11:52:33.000424 139822730319616 configuration_utils.py:319] Model config BertConfig {
  "_num_labels": 2,
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "bos_token_id": null,
  "decoder_start_token_id": null,
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": null,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "is_encoder_decoder": false,
  "label

In [113]:
def bert_text_preparation(text, tokenizer):
    """Preparing the input for BERT
    
    Takes a string argument and performs
    pre-processing like adding special tokens,
    tokenization, tokens to ids, and tokens to
    segment ids. All tokens are mapped to seg-
    ment id = 1.
    
    Args:
        text (str): Text to be converted
        tokenizer (obj): Tokenizer object
            to convert text into BERT-re-
            adable tokens and ids
        
    Returns:
        list: List of BERT-readable tokens
        obj: Torch tensor with token ids
        obj: Torch tensor segment ids
    
    
    """
    marked_text = "[CLS] " + text + " [SEP]"
    tokenized_text = tokenizer.tokenize(marked_text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1]*len(indexed_tokens)

    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])

    return tokenized_text, tokens_tensor, segments_tensors

def get_bert_embeddings(tokens_tensor, segments_tensors, model):
    """Get embeddings from an embedding model
    
    Args:
        tokens_tensor (obj): Torch tensor size [n_tokens]
            with token ids for each token in text
        segments_tensors (obj): Torch tensor size [n_tokens]
            with segment ids for each token in text
        model (obj): Embedding model to generate embeddings
            from token and segment ids
    
    Returns:
        list: List of list of floats of size
            [n_tokens, n_embedding_dimensions]
            containing embeddings for each token
    
    """
    
    # Gradient calculation id disabled
    # Model is in inference mode
    with torch.no_grad():
        outputs = model(tokens_tensor, segments_tensors)
        # Removing the first hidden state
        # The first state is the input state
        hidden_states = outputs[2][1:]

    # Getting embeddings from the final BERT layer
    token_embeddings = hidden_states[-1]
    # Collapsing the tensor into 1-dimension
    token_embeddings = torch.squeeze(token_embeddings, dim=0)
    # Converting torchtensors to lists
    list_token_embeddings = [token_embed.tolist() for token_embed in token_embeddings]

    return list_token_embeddings

In [129]:
texts = [
#     'They are a very trusting person.',
#     'They are a very trustworthy person.'
    'They are quick to trust people.',
    'They inspire trust from people.',
    'trusting',
    'trustworthy',
    'trusted'
]

In [313]:
text = "Mary trusts and has faith in James."

wois = ['James','Mary']

contexts = [
    "They are highly trusting of others.",
    'They are highly trusted by others.'
]

words = [
    'trusting',
    'trusted'
]

benchmark_text = 'This is a benchmark to get rid of frequency effects.'

In [319]:
#text = "John cites Mary as a source of his inspiration."
text = "Mary comforts and soothes James."

wois = ['James', 'Mary']

contexts = [
    "The blanket is comforting.",
    "The news is comforting.",
    "Hot tea is comforting.",
    ""
    'I feel comforted.'
]

words = ['comforting','comforted']

In [324]:
#text = "John cites Mary as a source of his inspiration."
text = "Mary inspires John."

wois = ['John','Mary']

contexts = [
    "He is inspiring to the next generation.",
    'I was inspired by his idea.'
]

words = ['inspiring','inspired']

In [295]:
#text = "John cites Mary as a source of his inspiration."
#text = "Mary bores John."
text = 'John is bored by Mary.'

wois = ['John','Mary']

contexts = [
    "He is bored by the movie.",
    'The movie is boring.'
]

words = ['bored','boring']

In [238]:
text = 'James finds John to be rather dull.'

wois = ['James','John']

words = [
    'boring',
    'bored'
]

In [ ]:
texts = ['The happy man ran home on hearing the news.',
         'The happy news caused the woman to run home immediately.']

In [385]:
def get_embeds(contexts,wois):
    target_word_embeddings = []
    for i,context in enumerate(contexts):
        tokenized_text, tokens_tensor, segments_tensors = bert_text_preparation(context, tokenizer)
        tokenized_word, _, _ = bert_text_preparation(wois[i], tokenizer)
        tokenized_word = tokenized_word[1]
        list_token_embeddings = get_bert_embeddings(tokens_tensor, segments_tensors, model)
        
        word_index = tokenized_text.index(tokenized_word)
        print(tokenized_text,tokenized_word,word_index)
        word_embedding = list_token_embeddings[word_index]
        target_word_embeddings.append(word_embedding)
    return target_word_embeddings

In [386]:
res = get_embeds(['He is very trusting of other people.','She is trusting and quick to confide her secrets.'],
          ['trusting','trusting'])

['[CLS]', 'he', 'is', 'very', 'trusting', 'of', 'other', 'people', '.', '[SEP]'] trusting 4
['[CLS]', 'she', 'is', 'trusting', 'and', 'quick', 'to', 'con', '##fide', 'her', 'secrets', '.', '[SEP]'] trusting 3


In [342]:
1 - cosine(res[0],res[1])

0.8046946142354843

In [339]:
res = get_embeds(['He is very trusted.','She is trusted and can be counted on to keep secrets.'],
          'trusted')

['[CLS]', 'he', 'is', 'very', 'trusted', '.', '[SEP]'] trusted 4
['[CLS]', 'she', 'is', 'trusted', 'and', 'can', 'be', 'counted', 'on', 'to', 'keep', 'secrets', '.', '[SEP]'] trusted 3


In [340]:
1 - cosine(res[0],res[1])

0.8453657171265186

In [325]:
# Getting embeddings for the target
# word in all given contexts
target_woi_embeddings, target_word_embeddings, benchmark_embedding = [], [], None

benchmark_tokenized, benchmark_tensor, benchmark_segments = bert_text_preparation(benchmark_text, tokenizer)
benchmark_token_embeddings = get_bert_embeddings(benchmark_tensor, benchmark_segments, model)
benchmark_embedding = benchmark_token_embeddings[benchmark_tokenized.index('frequency')]

for i,woi in enumerate(wois):
    tokenized_text, tokens_tensor, segments_tensors = bert_text_preparation(text, tokenizer)
    list_token_embeddings = get_bert_embeddings(tokens_tensor, segments_tensors, model)
    
    word_index = tokenized_text.index(woi.lower())
    print(i,woi,tokenized_text,word_index)
    
#     # Find the position 'bank' in list of tokens
#     if 'interest' in tokenized_text:
#         word_index = tokenized_text.index('editor')
#     else:
#         word_index = 0

    # Get the embedding for bank
    word_embedding = list_token_embeddings[word_index]
    target_woi_embeddings.append(word_embedding)
    
for i,context in enumerate(contexts):
    tokenized_text, tokens_tensor, segments_tensors = bert_text_preparation(context, tokenizer)
    list_token_embeddings = get_bert_embeddings(tokens_tensor, segments_tensors, model)
    
    #word_index = 1
    try:
        word_index = tokenized_text.index(words[i])
    except ValueError:
        word_index = tokenized_text.index(words[i].split('ed')[0])
    print(i,tokenized_text,words[i],word_index)
    word_embedding = list_token_embeddings[word_index]
    target_word_embeddings.append(word_embedding)

0 John ['[CLS]', 'mary', 'inspire', '##s', 'john', '.', '[SEP]'] 4
1 Mary ['[CLS]', 'mary', 'inspire', '##s', 'john', '.', '[SEP]'] 1
0 ['[CLS]', 'he', 'is', 'inspiring', 'to', 'the', 'next', 'generation', '.', '[SEP]'] inspiring 3
1 ['[CLS]', 'i', 'was', 'inspired', 'by', 'his', 'idea', '.', '[SEP]'] inspired 3


In [328]:
from scipy.spatial.distance import cosine

# Calculating the distance between the
# embeddings of 'bank' in all the
# given contexts of the word

list_of_distances = []
for woi, embed1 in zip(wois, target_woi_embeddings):
    for text2, embed2 in zip(words, target_word_embeddings):
        cos_dist = 1 - cosine(embed1, embed2)
        benchmark_dist = 1 - cosine(benchmark_embedding, embed1)
        list_of_distances.append([text + ' | ' + woi.upper(), text2, cos_dist, benchmark_dist])

distances_df = pd.DataFrame(list_of_distances, columns=['text1', 'text2', 'distance', 'dist(benchmark, text1)'])
distances_df

,text1,text2,distance,"dist(benchmark, text1)"
0,Mary inspires John. | JOHN,inspiring,0.349679,0.217879
1,Mary inspires John. | JOHN,inspired,0.290506,0.217879
2,Mary inspires John. | MARY,inspiring,0.285497,0.262546
3,Mary inspires John. | MARY,inspired,0.201510,0.262546


In [329]:
distances_df['adj_distance'] = distances_df['distance']/distances_df['dist(benchmark, text1)']
distances_df

,text1,text2,distance,"dist(benchmark, text1)",adj_distance
0,Mary inspires John. | JOHN,inspiring,0.349679,0.217879,1.604920
1,Mary inspires John. | JOHN,inspired,0.290506,0.217879,1.333337
2,Mary inspires John. | MARY,inspiring,0.285497,0.262546,1.087419
3,Mary inspires John. | MARY,inspired,0.201510,0.262546,0.767522


In [318]:
d_J_ing = 0.314680
d_J_ed = 0.308100
d_M_ing = 0.271551
d_M_ed = 0.300490
d_ing_bench = 0.280248
d_ed_bench = 0.292386
print(d_J_ing / d_J_ed) # J closer to trusting than trusted
print(d_M_ing / d_M_ed) # M closer to trusted than trusting
print(d_J_ing / d_M_ing) # trusting closer to John than Mary
print(d_J_ed / d_M_ed) # trusted closer to John than Mary

1.0213567023693606
0.9036939665213485
1.1588246775007274
1.0253253020067223


In [310]:
distances_df.loc[distances_df['text2']=='trusting'].sort_values('distance',ascending=False)

,text1,text2,distance
0,Mary trusts and has faith in James. | JAMES,trusting,0.314680
2,Mary trusts and has faith in James. | MARY,trusting,0.271551


In [311]:
distances_df.loc[distances_df['text2']=='trusted'].sort_values('distance',ascending=False)

,text1,text2,distance
1,Mary trusts and has faith in James. | JAMES,trusted,0.30810
3,Mary trusts and has faith in James. | MARY,trusted,0.30049


In [298]:
distances_df.loc[distances_df['text2']=='bored'].sort_values('distance',ascending=False)

,text1,text2,distance
0,Mary bores John. | JOHN,bored,0.247160
2,Mary bores John. | MARY,bored,0.245556


In [299]:
distances_df.loc[distances_df['text2']=='boring'].sort_values('distance',ascending=False)

,text1,text2,distance
1,Mary bores John. | JOHN,boring,0.239369
3,Mary bores John. | MARY,boring,0.197656


In [293]:
distances_df.loc[distances_df['text2']=='inspired'].sort_values('distance',ascending=False)

,text1,text2,distance
1,Mary inspires John. | JOHN,inspired,0.290506
3,Mary inspires John. | MARY,inspired,0.201510


In [294]:
distances_df.loc[distances_df['text2']=='inspiring'].sort_values('distance',ascending=False)

,text1,text2,distance
0,Mary inspires John. | JOHN,inspiring,0.349679
2,Mary inspires John. | MARY,inspiring,0.285497


In [2]:
BLACKLIST_WORDS = {'shit','shitty','fuck','crap','crappy','gore','trump','tree','sun','soil','shanghai','john',
                  'desert','turn','earthquake','lightning','hurricane','cyclone','tornado','storm','wildfire',
                  'fire','cool','warm','hot','human','earth','planet','global','climate','change','cold'}
PATH_TO_EMOLEX = "/u/scr/yiweil/datasets/NRC-Emotion-Intensity-Lexicon-v1.txt"
EMOLEX = pd.read_csv(PATH_TO_EMOLEX,sep='\t')
EMOLEX = EMOLEX.loc[~EMOLEX['word'].isin(BLACKLIST_WORDS)]
EMOLEX_EMOS = EMOLEX['emotion'].value_counts().index
print(EMOLEX.head())

print(EMOLEX[
    'emotion'
].value_counts())

        word emotion  emotion-intensity-score
0   outraged   anger                    0.964
1  brutality   anger                    0.959
2     hatred   anger                    0.953
3    hateful   anger                    0.940
4  terrorize   anger                    0.939
fear            1754
trust           1562
anger           1475
sadness         1292
joy             1265
disgust         1085
anticipation     862
surprise         578
Name: emotion, dtype: int64


In [413]:
high_words_per_emotion = {
    emo: EMOLEX.loc[(EMOLEX['emotion'] == emo) &
                    (EMOLEX['emotion-intensity-score']>=0.5)]['word'].values
    for emo in EMOLEX_EMOS}

In [417]:
nlps_per_emotion, vectors_per_emotion = defaultdict(dict), defaultdict(dict)
for emo in EMOLEX_EMOS:
    for w in high_words_per_emotion[emo]:
        nlps_per_emotion[emo][w] = nlp(w)
        vectors_per_emotion[emo][w] = nlps_per_emotion[emo][w].vector

In [462]:
seed_embeds_per_emotion = {
    'trust': {'exp': get_embeds(['He is very trusting of other people.',
                                       'She is trusting and quick to confide her secrets.',
                                       'Sometimes I am too trusting of strangers.',
                                      'They are not very trusting and somewhat skeptical of the news.'],
          ['trusting']*4),
              'stim':get_embeds(['He is very trusted.',
                                        'She is trusted and can be counted on to keep secrets.',
                                       'The information comes from a trusted source.',
                                       'The conclusions are robust and can be trusted.',
                                       'He is very trustworthy.',
                                       'She is trustworthy and can be depended on to keep secrets.',
                                       'The information comes from a trustworthy and reliable source.',
                                       'The conclusions are robust and trustworthy.'],
          ['trusted']*4+['trustworthy']*4)},
    'fear': {'exp': get_embeds(['He is very fearful of other people.',
                                       'She is fearful and easily frightened.',
                                       'I am really scared of spiders.',
                                      'They are easily scared by horror movies.'],
          ['fearful','frightened','scared','scared']),
             'stim': get_embeds(['He is highly feared as a mafioso.',
                                        'She is feared and can make you tremble in fright.',
                                       'The professor was feared by all his students.',
                                       'They are formidable and fearsome opponents.'],
          ['feared','feared','feared','fearsome'])
            },
    'anger': {'stim': get_embeds(['The child is maddening and makes me so frustrated.',
                                'This customer is so annoying.',
                                'The news is really infuriating and has me extremely provoked.'],
                                ['maddening','annoying','infuriating']),
              'exp': get_embeds(['This child makes me so mad and frustrated',
                               'This customer is really angry at the clerk.',
                               'I am furious at the anger-provoking news.'],
                               ['mad','angry','furious'])
             },
    'sadness': {'stim': get_embeds(['The story was truly tragic and moved the audience to tears.',
                                  'It is so saddening to see children crying.',
                                  'The sad movie left the audience weeping.',
                                  'The sad news completely erased her good mood.'],
                                  ['tragic','saddening','sad','sad']),
              'exp': get_embeds(['The audience was sad and tearful by the end of the tragic story.',
                               'The children were extremely sad and sobbing loudly.',
                               'Many audience members were sad to the point of weeping after watching the tragedy.',
                               'Her good mood turned sad after she heard about the accident.'],
                               ['sad','sad','sad','sad'])
             },
    'joy': {'stim': get_embeds(['He was extremely happy after hearing the joyous news.',
                               'The happy encounter left her elated and jumping for joy.',
                              'The heartening story cheered her up significantly.'],
                              ['joyous','happy','heartening']),
              'exp': get_embeds(['He was extremely happy after hearing the joyous news.',
                               'She was elated and jumping for joy to see her boyfriend after a long time.',
                               'The birthday surprise left her gladdened and filled her with happiness'],
                               ['happy','elated','gladdened'])
             },
    'disgust': {'stim': get_embeds(['The rotten and moldy food is absolutely disgusting.',
                                  'This kind of gross behavior is rightfully considered disgusting.',
                                  'The smell is disgusting and makes me want to puke.'],
                                  ['disgusting','disgusting','disgusting']),
              'exp': get_embeds(['He was disgusted by the mold on his food.',
                               "She felt disgusted by the man's gross behavior",
                               'I am so disgusted that I want to puke'],
                               ['disgusted','disgusted','disgusted'])
             },
    'anticipation': {'exp': get_embeds(['The eager fans are nervously anticipating the game.',
                                       "Readers are eagerly anticipating the new book, as evidenced by record-shattering pre-orders.",
                                       'Fans of the franchise have been anticipating the sequel for years.',
                                       'We are struggling to be patient in awaiting the outcome.'],
                                       ['anticipating','anticipating','anticipating','awaiting']),
              'stim': get_embeds(['The game tomorrow night will be highly anticipated by eager fans.',
                               "The novelist's next book is highly anticipated, with pre-orders setting new records.",
                               'The eagerly awaited sequel finally comes to theaters this month.',
                               'The outcome had been long anticipated.'],
                               ['anticipated','anticipated','awaited','anticipated'])
             },
    'surprise': {'stim': get_embeds(['The surprising turn of events left the crowd speechless.',
                                   'The unexpected ending took the audience by surprise.',
                                   'The proposal was surprising, since they had been dating for only two months.',
                                   'The experience can be surprising, going against your initial expectations.'],
                                   ['surprising','unexpected','surprising','surprising']),
              'exp': get_embeds(['The crowd was left speechless and utterly surprised.',
                               'The audience was completely surprised by the unexpected ending.',
                               'She was surprised by his proposal, since they had been dating for only two months.',
                               'We were surprised by the unfamiliar experience, which went against our initial expectations.'],
                               ['surprised','surprised','surprised','surprised'])
             },}

['[CLS]', 'he', 'is', 'very', 'trusting', 'of', 'other', 'people', '.', '[SEP]'] trusting 4
['[CLS]', 'she', 'is', 'trusting', 'and', 'quick', 'to', 'con', '##fide', 'her', 'secrets', '.', '[SEP]'] trusting 3
['[CLS]', 'sometimes', 'i', 'am', 'too', 'trusting', 'of', 'strangers', '.', '[SEP]'] trusting 5
['[CLS]', 'they', 'are', 'not', 'very', 'trusting', 'and', 'somewhat', 'skeptical', 'of', 'the', 'news', '.', '[SEP]'] trusting 5
['[CLS]', 'he', 'is', 'very', 'trusted', '.', '[SEP]'] trusted 4
['[CLS]', 'she', 'is', 'trusted', 'and', 'can', 'be', 'counted', 'on', 'to', 'keep', 'secrets', '.', '[SEP]'] trusted 3
['[CLS]', 'the', 'information', 'comes', 'from', 'a', 'trusted', 'source', '.', '[SEP]'] trusted 6
['[CLS]', 'the', 'conclusions', 'are', 'robust', 'and', 'can', 'be', 'trusted', '.', '[SEP]'] trusted 8
['[CLS]', 'he', 'is', 'very', 'trust', '##worthy', '.', '[SEP]'] trust 4
['[CLS]', 'she', 'is', 'trust', '##worthy', 'and', 'can', 'be', 'depended', 'on', 'to', 'keep', 'secret

In [463]:
seed_sims_per_emo = defaultdict(lambda: defaultdict(dict))
for emo in EMOLEX_EMOS:
    for seed_cat in ['exp','stim']:
        for w in high_words_per_emotion[emo]:
            seed_sims_per_emo[emo][seed_cat][w] = sum([(1-cosine(seed_embed, vectors_per_emotion[emo][w]))
                                                  for seed_embed in seed_embeds_per_emotion[emo][seed_cat]])

In [464]:
ratios_exp_stim_per_emo = {emo: {w: (seed_sims_per_emo[emo]['exp'][w]/len(seed_embeds_per_emotion[emo]['exp']))/\
                               (seed_sims_per_emo[emo]['stim'][w]/len(seed_embeds_per_emotion[emo]['stim']))
                               for w in high_words_per_emotion[emo]}
                         for emo in EMOLEX_EMOS}

In [461]:
', '.join([x[0] for x in sorted(ratios_ed_ing_per_emo['trust'].items(), key=lambda x: x[1], reverse=True)[-40:]])

'befriend, pleasant, apologetic, generosity, timing, admiration, ally, agreeing, proud, amour, bless, kind, complimentary, intelligent, guide, eager, bride, protective, forgive, admitting, bloom, gospel, considerate, lovable, nerds, forgiving, inviting, generous, kindred, ourselves, humble, compliment, instructions, cuddle, praise, trusting, homage, innocent, temperate, romantic'

In [458]:
', '.join([x[0] for x in sorted(ratios_ed_ing_per_emo['fear'].items(), key=lambda x: x[1], reverse=True)[:25]])

'perish, combatant, cemetery, crippling, appalling, insurmountable, hellish, bloodshed, assail, grisly, perpetrator, soscared, harrowing, incrimination, wrath, irreparable, horrid, reprisal, perdition, annihilation, murderer, plunder, upheaval, nefarious, angina'

In [456]:
', '.join([x[0] for x in sorted(ratios_ed_ing_per_emo['anger'].items(), key=lambda x: x[1], reverse=True)[-25:]])

'prejudicial, atrocious, overbearing, blasphemous, manslaughter, unforgiving, intolerant, aggressor, coercion, depraved, mutilation, barbaric, intolerable, morbidity, inexcusable, vexed, monstrosity, intrusive, condescension, persecute, irreconcilable, disrespectful, depravity, extermination, crucifixion'

In [454]:
', '.join([x[0] for x in sorted(ratios_ed_ing_per_emo['sadness'].items(), key=lambda x: x[1], reverse=True)[:25]])

'homeless, unwell, teary, comatose, defenseless, departed, upset, disillusionment, urn, disgruntled, defeated, injured, demoralized, displeased, unhappy, grieving, battered, disfigured, disabled, decayed, bereaved, tearful, destitute, dumps, homesick'

In [452]:
', '.join([x[0] for x in sorted(ratios_ed_ing_per_emo['joy'].items(), key=lambda x: x[1], reverse=True)[-25:]])

'diamond, passionate, humor, intimately, fortunes, mastery, treat, aspiring, sweets, darling, special, cruising, leisure, dayoff, achieved, wellness, princely, bloom, therapeutic, entertainment, achievement, holidays, awards, winner, award'

In [450]:
', '.join([x[0] for x in sorted(ratios_ed_ing_per_emo['disgust'].items(), key=lambda x: x[1], reverse=True)[:25]])

'sump, barf, retard, cringe, enslaved, flea, loo, disgust, wench, bitterly, repellant, aghast, cess, repulsion, hag, revulsion, resentment, brawl, gob, canker, shame, remains, outhouse, dank, vulture'

In [448]:
', '.join([x[0] for x in sorted(ratios_ed_ing_per_emo['anticipation'].items(), key=lambda x: x[1], reverse=True)[-25:]])

'strive, enthusiasm, exigent, thriving, grow, develop, warn, hankering, owing, contingent, sneaking, cue, tease, grasping, vacation, thrill, readiness, accelerate, puppy, worrying, enjoying, feeling, forming, young, compensate'

In [445]:
', '.join([x[0] for x in sorted(ratios_ed_ing_per_emo['surprise'].items(), key=lambda x: x[1], reverse=True)[:25]])

'awestruck, outcry, blitz, bewildered, rejoice, bewilderment, flinch, paralyzed, aghast, amazement, frightened, jubilant, strangle, astonishment, murderous, surprised, revolt, amazedness, breathless, excited, gasp, dismay, bomb, awe, thunderstruck'

In [465]:
pickle.dump(ratios_exp_stim_per_emo,open('ratios_exp_stim_per_emo.pkl','wb'))

In [398]:
trustworthy_words = [w for w in ratio_trustworthy_trusting if ratio_trustworthy_trusting[w] >= 1]
trusting_words = [w for w in ratio_trustworthy_trusting if ratio_trustworthy_trusting[w] < 1]
len(trustworthy_words),len(trusting_words)

(450, 379)

In [395]:
ratio_trustworthy_trusting['scientist']

1.0039665078711244

In [397]:
ratio_trustworthy_trusting['trustworthy']

0.9812272847585687

In [ ]:
# unquestionably, unquestionable, truthful, truthfulness, soundness, proficiency, competent, correctness, 

In [355]:
seed_sims['trustworthy']['faith'],seed_sims['trusting']['faith']

(0.33895293176166386, 0.3990370391627408)

In [346]:
seed_sims['trustworthy']['convincing'],seed_sims['trusting']['convincing']

(0.2868058417195176, 0.23842786730440946)

In [347]:
seed_sims['trustworthy']['expert'],seed_sims['trusting']['expert']

(0.27545600616534915, 0.23470223817721514)

In [348]:
seed_sims['trustworthy']['scientific'],seed_sims['trusting']['scientific']

(0.2221806177175083, 0.18040295915752946)

In [349]:
seed_sims['trustworthy']['team'],seed_sims['trusting']['team']

(0.23006445235912099, 0.20640873045769803)

In [350]:
seed_sims['trustworthy']['credibility'],seed_sims['trusting']['credibility']

(0.2812008071995461, 0.23671180939025604)

In [351]:
seed_sims['trustworthy']['loyalty'],seed_sims['trusting']['loyalty']

(0.4615678234249321, 0.49477126198217425)

In [352]:
seed_sims['trustworthy']['friend'],seed_sims['trusting']['friend']

(0.3428814107521706, 0.3340188495347566)

In [354]:
seed_sims['trustworthy']['confide'],seed_sims['trusting']['confide']

(0.5043866783085269, 0.5573110484678789)

In [125]:
from sklearn import cluster, metrics
NUM_CLUSTERS = 2
X = np.asarray(list(trust_vectors.values()))
print(X.shape)
kmeans = cluster.KMeans(n_clusters=NUM_CLUSTERS)
kmeans.fit(X)
 
labels = kmeans.labels_
centroids = kmeans.cluster_centers_
 
print ("Cluster id labels for inputted data")
print (labels)
print ("Centroids data")
print (centroids)
 
print ("Score (Opposite of the value of X on the K-means objective which is Sum of distances of samples to their closest cluster center):")
print (kmeans.score(X))
 
silhouette_score = metrics.silhouette_score(X, labels, metric='euclidean')
 
print ("Silhouette_score: ")
print (silhouette_score)

(1562, 768)
Cluster id labels for inputted data
[0 1 1 ... 1 1 0]
Centroids data
[[-0.17448919 -1.272423   -0.62499315 ... -0.13562134 -0.43684334
  -0.2900922 ]
 [ 0.32006335 -0.14674556 -0.555922   ...  0.01643606 -0.49558103
  -0.12312236]]
Score (Opposite of the value of X on the K-means objective which is Sum of distances of samples to their closest cluster center):
-534307.7
Silhouette_score: 
0.39945617


In [126]:
w2cluster = {trust_words[i]: labels[i] for i in range(len(trust_words))}
df = pd.DataFrame({
    'word': trust_words,
    'cluster': labels
})
df['cluster'].value_counts()

1    1295
0     267
Name: cluster, dtype: int64

In [127]:
df.loc[df['cluster']==0]['word'].values

array(['truthfulness', 'truthful', 'entrust', 'safekeeping', 'admirable',
       'reassurance', 'heartfelt', 'credential', 'virtuous', 'motherhood',
       'friendliness', 'skillful', 'approving', 'agreeable',
       'indestructible', 'justifiable', 'monogamy', 'confide',
       'commendable', 'indivisible', 'diligence', 'affirm', 'sanctify',
       'cohesive', 'unwavering', 'benevolence', 'attestation',
       'impeccable', 'steadfast', 'constancy', 'matrimony', 'lovable',
       'advisable', 'competency', 'unconditionally', 'conscientious',
       'durability', 'coexist', 'credence', 'devotional',
       'sanctification', 'authorize', 'fireman', 'positivity', 'certify',
       'purify', 'commend', 'unimpeachable', 'attentive', 'corroborate',
       'absolution', 'considerate', 'magnificence', 'babysitter',
       'humility', 'serenity', 'unconditional', 'unquestionable',
       'mainstay', 'transcendence', 'befriend', 'enlighten', 'eminence',
       'vigilant', 'tranquility', 'substa

In [128]:
df.loc[df['cluster']==1]['word'].values

array(['trusted', 'trustworthy', 'truth', ..., 'divorce', 'mistakes',
       'bait'], dtype=object)

In [35]:
df.loc[df['cluster']==2]['word'].values

array(['confidentially', 'authenticate', 'kindred', 'cooperating',
       'reverence', 'affirmatively', 'guidebook', 'warranty', 'mamma',
       'endorse', 'fortitude', 'aspiration', 'saintly', 'brotherly',
       'approvement', 'cherish', 'praiseworthy', 'mediator', 'notary',
       'vetted', 'brighten', 'complimentary', 'sobriety', 'helper',
       'revere', 'seniority', 'godly', 'bylaw', 'checklist', 'attest',
       'adore', 'legalized', 'watchdog', 'connective', 'cuddle',
       'adhering', 'curable', 'mediate', 'invocation', 'doubtless',
       'answerable', 'verily', 'candid', 'edification', 'captivate',
       'benedictory', 'reverie', 'admissible', 'savor', 'vouch',
       'forgiving', 'undying', 'pontiff', 'backer', 'aspire', 'hosannah',
       'endow', 'crowning', 'laudation', 'votive', 'putative', 'partake',
       'chairwoman', 'laud', 'journeyman', 'foresee', 'dictum', 'enliven',
       'mlk', 'villager', 'conformity', 'merci', 'governess', 'atone',
       'sentry', 'brid

In [36]:
df.loc[df['cluster']==3]['word'].values

array(['truthfulness', 'trustworthy', 'truthful', 'entrust', 'sisterhood',
       'reputable', 'wholesome', 'reassurance', 'heartfelt', 'credential',
       'creditable', 'motherhood', 'guardianship', 'friendliness',
       'nurture', 'skillful', 'approving', 'agreeable', 'justifiable',
       'solidity', 'confide', 'affirm', 'faultless', 'cohesive',
       'harmoniously', 'lovable', 'advisable', 'competency', 'coexist',
       'soundness', 'devotional', 'authorize', 'positivity', 'surety',
       'considerate', 'humility', 'serenity', 'mainstay', 'transcendence',
       'eminence', 'joyful', 'firmness', 'angelic', 'coexisting',
       'cheerfulness', 'insure', 'punctual', 'determinate', 'frankness',
       'vitality', 'cohesion', 'uplift', 'lovemaking', 'spotless',
       'pertinent', 'concordance', 'restorative', 'foresight', 'coalesce',
       'congruence', 'greatness', 'meditate', 'tantamount', 'inimitable',
       'immovable', 'antidote', 'regularity', 'stamina', 'chastity',
     

In [37]:
df.loc[df['cluster']==4]['word'].values

array(['safekeeping', 'fireman', 'purify', 'babysitter', 'befriend',
       'husbandry', 'counsellor', 'matron', 'betrothed', 'midwife',
       'cultivate', 'eyewitness', 'inoculation', 'watchman',
       'immunization', 'connoisseur', 'obstetrician', 'housewife',
       'custodian', 'beautification', 'radiance', 'proverbs',
       'countryman', 'calculator', 'cashier', 'bridesmaid', 'strategist',
       'tradesmen', 'conveyancing', 'arbitrator', 'comptroller',
       'elucidate', 'antiseptic', 'shopkeeper', 'preservative',
       'amortization', 'dermatologist', 'curfew', 'bursary', 'bailiff',
       'knickers', 'boomerang', 'spaniel', 'prophylactic', 'thermometer',
       'antifungal', 'wadsworth', 'acrobat', 'sceptical', 'bieber',
       'scoundrel'], dtype=object)

In [38]:
df.loc[df['cluster']==5]['word'].values

array(['monogamy', 'indivisible', 'sanctify', 'attestation', 'constancy',
       'matrimony', 'credence', 'sanctification', 'certify', 'commend',
       'corroborate', 'absolution', 'enlighten', 'substantiate',
       'unquestioned', 'everlasting', 'corroboration', 'pedigree',
       'applaud', 'exalt', 'rejoice', 'conciliation', 'uncensured',
       'exalted', 'fraternal', 'approbation', 'adoration', 'liberate',
       'perpetuity', 'amen', 'accolade', 'indelible', 'laudatory',
       'approbate', 'preponderance', 'oblige', 'exaltation', 'indemnity',
       'bridegroom', 'bequest', 'occupant', 'reconsideration', 'eulogize',
       'supplication', 'glorify', 'reimbursement', 'executor', 'parietal',
       'eulogy', 'eulogistic', 'plaudit', 'reparation', 'disclaim',
       'eulogium', 'crescendo', 'presumption', 'insolvent', 'ejaculation'],
      dtype=object)

In [39]:
df.loc[df['cluster']==6]['word'].values

array(['trusted', 'truth', 'honor', 'honest', 'trusting', 'brotherhood',
       'credibility', 'integrity', 'honorable', 'committed', 'trust',
       'faithful', 'sincere', 'true', 'partnership', 'responsible',
       'wisdom', 'cooperative', 'vow', 'oath', 'credible', 'respectable',
       'promise', 'verified', 'hero', 'respected', 'compassion',
       'approval', 'loyal', 'competence', 'ally', 'respect', 'advised',
       'safeguard', 'authentic', 'genuine', 'friend', 'stable',
       'friendships', 'respects', 'faith', 'proven', 'accepting',
       'unification', 'confirmation', 'protecting', 'supporter',
       'respectful', 'fact', 'believing', 'confident', 'partners',
       'agreement', 'excellence', 'equality', 'virtue', 'honored',
       'closeness', 'allegiance', 'determination', 'straightforward',
       'guarantee', 'morality', 'admire', 'admiration', 'assurance',
       'mentor', 'innocent', 'reliable', 'prepared', 'sanctuary',
       'mother', 'fulfilled', 'compassionate